# Cell 1 — Imports and basic utilities

This cell imports the primary libraries and utilities required by the automation. 
It establishes the foundational tools for database connectivity, data processing, time handling, file operations, and notebook output formatting.

Purpose:
- Provide database drivers and engine support (`pymysql`, `sqlalchemy`) for connecting to MySQL.
- Load `pandas` for data manipulation and `datetime`/`time` for date/time logic and performance timing.
- Include `os` for filesystem operations and `IPython.display` helpers for improved notebook feedback.

Why this matters:
- Centralizes dependency declarations so subsequent sections can rely on consistent APIs for querying, transforming, and exporting data.


In [1]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import time as tm
from IPython.display import display, Markdown
from datetime import datetime, timedelta
import os


# Cell 2 — Helper: formatted output (printmd)

Defines a helper function that prints formatted Markdown messages for cleaner notebook output.

Purpose:
- Provide structured and readable runtime messages.
- Differentiate normal logs from warnings or alerts using Markdown formatting.

Why this matters:
- Improves traceability and readability during manual or automated runs.

                                                                

In [2]:
# Function to print colored text in Jupyter Notebook
def printmd(string, color="black"):
    display(Markdown(f"<span style='color:{color}; font-size:16px;'>{string}</span>"))

# Cell 3 — Timer start / runtime logging

Begins execution timing and logs the query start time.

Purpose:
- Capture when the automation started running.
- Initialize performance tracking.

Why this matters:
- Useful for debugging and performance evaluation, especially during scheduled or batch runs.


In [3]:
# Start timer
query_start = tm.time()
printmd(f"Query Start Run at: {tm.asctime(tm.localtime(query_start))}", "green")

<span style='color:green; font-size:16px;'>Query Start Run at: Wed Oct  1 12:44:50 2025</span>

### 💻 Dynamic Username Extraction — PC Path Handling

This code block dynamically determines the current computer’s username folder based on the working directory path.  
It’s a crucial setup step for making the script **portable and adaptive** to any user environment, ensuring that file paths referencing local directories (like templates, reports, or exports) are automatically configured without manual editing.

**Purpose:**
To detect the logged-in user’s folder name on the local machine and store it in the variable `pc_username_folder`, allowing file path automation later in the script.

**How It Works:**
1. **Import the `os` Module** – Provides functions to interact with the operating system, particularly for handling directories and paths.  
2. **Get Current Working Directory** – `os.path.abspath(os.getcwd())` retrieves the absolute path of the script’s current directory.  
3. **Slice Path After `C:\Users\`** – Calculates where the username starts in the path string using `len("c:\\users\\")`.  
4. **Locate the Next Folder Separator** – Finds the next backslash `\` to determine where the username ends.  
5. **Extract the Username** – Slices out the username portion and assigns it to `pc_username_folder`.  
6. **Print the Username** – Displays the detected username for verification and debugging purposes.

**Example in Workflow:**
The extracted username (`pc_username_folder`) is later used to build file paths dynamically,  
for example:


In [4]:
#🔍 Locating the directory/folder name after C:\Users\(x)\
#** (then adding the default directory of automation)

# first = len("c:\\users\\")
# print(os.path.abspath(os.getcwd()))
# pwd = os.path.abspath(os.getcwd())
# second = pwd[first:len(pwd)]
# third = second.find("\\")
# pc_username_folder =os.getenv('USERNAME')  # More reliable method than = second[0:third]
# print("username: "+ pc_username_folder)

# import os
first = len("c:\\users\\")
print(os.path.abspath(os.getcwd()))
pwd = os.path.abspath(os.getcwd())
second = pwd[first:len(pwd)]
third = second.find("\\")
pc_username_folder = second[0:third]
print("username: "+ pc_username_folder)


C:\Users\tripl\automation_project\sorted_folder\smash\Smash
username: tripl


### 🧭 Retrieving Stored Variables — Using `%store -r`

This code block retrieves key variables previously saved in the Jupyter environment using the `%store` magic command.  
It ensures that values set in previous runs or notebooks (such as dates, file paths, and client identifiers) are automatically reloaded and ready for use, maintaining workflow continuity across multiple executions.

**Purpose:**
To restore essential parameters like date ranges, export filenames, and client details, which are required for the automation to function consistently across sessions.

**How It Works:**
1. **`%store -r` Command** – The `-r` flag stands for “restore”. It retrieves a variable stored in Jupyter’s persistence system.  
2. Each line restores a specific variable that was saved earlier using `%store variable_name`.  
3. This allows the notebook to retain context even after being closed and reopened, without reinitializing values manually.

**Variables Restored:**
- **`start_date`** → Beginning of the extraction period  
- **`end_date`** → End of the extraction period  
- **`export_File`** → File path for storing the extracted results  
- **`filenamedt`** → Dynamically generated filename string (usually contains start and end date info)  
- **`client`** → Client identifier for which the extraction is being performed  
- **`row`** → Excel row number reference or insertion point in the template

**Example in Workflow:**
This block is typically run before any data extraction or saving logic.  
It ensures all downstream functions—like querying the database or saving to Excel—have access to the correct runtime variables set earlier in your automation.


In [5]:
%store -r start_date
%store -r end_date
%store -r export_File
%store -r filenamedt
%store -r client
%store -r row

no stored variable or alias client
no stored variable or alias row


In [6]:
# #💾 Save Location of excel file output
# folder_path = fr'C:\Python\SMASH'

In [7]:
#location of the template in your computer¶
#** make sure to place the template in this location
# Define file paths
# import_template = fr'C:\Users\{pc_username_folder}\automation_project\sorted_folder\Templates\V8 Templates\SMASH_Template_DBGL.xlsx'
# print(import_template)

### 🔐 Loading Login Credentials — Game Back Office Access

This cell runs an external notebook containing secure login credentials and retrieves the necessary credential data for authentication.  
It modularizes sensitive information (like usernames, passwords, or tokens) by storing them in a separate notebook, keeping the main automation cleaner and more secure.

**Purpose:**
To execute a supporting notebook (`logincreds_gameBO_smash.ipynb`) that contains credential setup logic,  
and restore the stored credential object (`smashcreds`) for use in login or data extraction processes.

**How It Works:**
1. **`%run` Magic Command** – Executes another Jupyter Notebook file (`logincreds_gameBO_smash.ipynb`) as if its contents were part of this notebook.  
   - This allows credentials or setup configurations to be defined externally and reused across multiple scripts.  
2. **`%store -r smashcreds`** – Restores the saved credential object (`smashcreds`) that was previously stored using `%store`.  
   - This variable typically includes information like login URLs, usernames, and encrypted or masked passwords.

**Example in Workflow:**
This step is executed before initiating any web automation or API connections that require authentication.  
By separating credentials into another notebook, the workflow maintains:
- **Security** — Sensitive data isn’t exposed directly in the main script.  
- **Modularity** — Credentials can be updated independently without altering the automation logic.  
- **Reusability** — The same credentials can be referenced by multiple automation scripts.



In [8]:
%run ./logincreds_gameBO_smash.ipynb
%store -r smashcreds

Stored 'smashcreds' (dict)


### ⏱️ Start Timer — Query Execution Tracking

This cell initializes a timer to monitor the duration of the data extraction process.  
Tracking query runtime helps in performance optimization and provides visibility into how long each part of the automation takes to execute.

**Purpose:**
To record the exact timestamp when the query process begins, enabling the user to calculate total execution time later for reporting or troubleshooting.

**How It Works:**
1. **`query_start = tm.time()`** – Captures the current time in seconds since the epoch (Unix timestamp) using the `time` module, and stores it in the variable `query_start`.  
2. **`tm.asctime(tm.localtime(query_start))`** – Converts the timestamp into a human-readable date and time format.  
3. **`printmd()`** – A custom or helper function designed to print styled text (in this case, green) within Jupyter Markdown cells for better readability.  
4. Displays a message showing exactly when the query execution started.

**Example in Workflow:**
This timer serves as the **starting point** for runtime measurement.  
At the end of the automation, another timestamp (`query_end`) will be captured, and the elapsed time (`query_end - query_start`) will show how long the query or extraction process took to complete.

**Best Practice:**
Including start and end timestamps in long-running scripts helps:
- Identify slow queries or network delays  
- Benchmark improvements after optimization  
- Provide transparency when reviewing automation logs


In [9]:
# Start timer
query_start = tm.time()
printmd(f"Query Start Run at: {tm.asctime(tm.localtime(query_start))}", "green")

<span style='color:green; font-size:16px;'>Query Start Run at: Wed Oct  1 12:44:50 2025</span>

In [10]:
#!pip install pymysql pandas sqlalchemy

### 🎮 Game Provider Configuration — IDs and Data Range Setup

This section defines the key parameters for identifying which game providers’ data will be extracted and the specific timeframe for the query.  
It acts as a **setup cell** that determines the scope of data retrieval within the automation process.

**Purpose:**
To specify:
- The **list of game provider IDs** whose logs will be queried from the database.  
- The **target database table** containing the gameplay data.  
- The **optional time range** (`start_date`, `end_date`) that constrains the query results to a specific period.

**How It Works:**
1. **`gpid_list`** – A Python list containing multiple game provider IDs as strings.  
   - Each ID corresponds to a specific provider whose transaction or game logs will be retrieved.  
   - The script later loops through this list to run queries or data validations for each provider automatically.  
2. **`table`** – Sets the name of the database table to be queried (e.g., `"gaming_data"`).  
   - Keeping it in a variable allows easy modification if the table name changes or needs to be reused in SQL queries.  
3. **`start_date` / `end_date` (commented)** – These variables define the query’s time range.  
   - Although commented out here, they are dynamically loaded earlier from `%store` commands or another date logic function.

**Example in Workflow:**
This block is executed before the main data extraction loop.  
The script will use these variables to:
- Dynamically build SQL queries,  
- Filter data per provider ID, and  
- Ensure only logs within the defined date range are processed.

**Best Practice:**
Keep provider IDs centralized in a list like this — it improves scalability and avoids hardcoding IDs across multiple queries.  
If new providers are added, they can be included here without modifying deeper parts of the script.


In [11]:
# List of game provider IDs
gpid_list = ['6540','6516','6582','6479','6517','6575','6519','5936','6580','6578','6527','6581',           
  '6577','6483','6528','6520','6482','6526','6515','6487','6621','6512','6579','6584','6518','6620',
  '6525','6622','6524','6522','6576','6529','6583','6514','6513','6523']

# Define table and time range
table = "gaming_data"
# start_date = "2025-09-01 00:00:00"
# end_date = "2025-09-30 23:59:59"


### 🗓️ Date Conversion, Range Adjustment, and Dynamic Filename Generation

This section converts stored date strings into datetime objects, adjusts the extraction time range,  
and dynamically generates filenames that include start/end dates and time zone identifiers.  
It ensures that both query parameters and output filenames remain consistent, traceable, and automatically aligned with the reporting period.

**Purpose:**
To handle all date-related processing and to create a unique, timestamped filename for saving the extracted game logs.

**How It Works:**

1. **Convert Date Strings to Datetime Objects**
   - `datetime.strptime()` converts the stored `start_date` and `end_date` strings (`YYYY-MM-DD HH:MM:SS`) into datetime objects for manipulation.

2. **Adjust the End Date Range**
   - Adds one day (`timedelta(days=1)`) and resets time to midnight (`.replace(hour=0, minute=0, second=0)`),  
     ensuring that the full range through the final date is included in the extraction.

3. **Reformat for Query Use**
   - Converts `end_dt` back to a formatted string (`'%Y-%m-%d %H:%M:%S'`) for SQL query integration.
   - `start_t` and `end_t` become ready-to-use query parameters.

4. **Prepare Date Strings for Filename Usage**
   - Reformats both dates (`st_dt` and `ed_dt`) to `YYYY-MM-DD` for cleaner file naming.
   - Extracts **month and day portions** (`%b` and `%d`) to form compact start and end identifiers like `Jan31-Feb01`.

5. **Append Time Zone Identifiers**
   - Iterates through the list `tzgroup = ["+8", "0", "-3"]` and concatenates them into a suffix (e.g., `_+8_0_-3`),  
     indicating multiple time zones covered by the report.

6. **Construct the Dynamic Filename**
   - Combines start name, end name, and timezone suffix → e.g., `Jan31-Feb01_+8_0_-3`
   - Stores this value in `filenamedt` using `%store` for reuse in other cells or notebooks.

7. **Generate Export File Path**
   - Defines `export_File` as the final Excel output path, e.g.:
     ```
     C:\Python\smash\smash_Jan31-Feb01_+8_0_-3.xlsx
     ```
   - Stores the path with `%store` for consistent access in later parts of the workflow.

8. **Print Outputs for Verification**
   - Displays both the generated filename and full export path for confirmation.

**Example in Workflow:**
This cell prepares standardized filenames and ensures the correct extraction window is used when pulling game data from the database.  
It helps in organizing output files by date and time zone, avoiding overwrites and simplifying archival management.

**Best Practice:**
- Always ensure `start_date` and `end_date` are in `YYYY-MM-DD HH:MM:SS` format to prevent parsing errors.  
- Consider externalizing the timezone list (`tzgroup`) if reports often vary by region.


In [12]:
# Convert string dates to datetime objects
start_dt = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_dt = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

# Add one day to the end date
end_dt += timedelta(days=1)
end_dt = end_dt.replace(hour=0, minute=0, second=0)

# Format the end date back to the desired string format
start_t = start_dt
end_t = end_dt.strftime('%Y-%m-%d %H:%M:%S')

#for filename use and saving to excel
ed_dt_1 = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
st_dt_1 = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

st_dt = datetime.strftime(st_dt_1, '%Y-%m-%d')  # ex.2024-01-31
ed_dt = datetime.strftime(ed_dt_1, '%Y-%m-%d')

# start date name
month_s = st_dt_1.strftime('%b')  # month word format
day_s = st_dt_1.strftime('%d')
str_name = month_s + day_s

# end date name
month_e = ed_dt_1.strftime('%b')  # month word format
day_e = ed_dt_1.strftime('%d')
end_name = month_e + day_e

# append time zone inclusion on name
tzgroup = ["+8","0","-3"]
tzname = ""
for ele in tzgroup:
    tzname += "_"
    tzname += ele

filenamedt = str_name + "-" + end_name + tzname
%store filenamedt

#export_File = fr'C:\Python\smash\smashgldbextract_{filenamedt}.xlsx'
export_File = fr'C:\Python\smash\smash_{filenamedt}.xlsx'
%store export_File
save_File = export_File

print(filenamedt)
print(export_File)

Stored 'filenamedt' (str)
Stored 'export_File' (str)
Sep01-Sep30_+8_0_-3
C:\Python\smash\smash_Sep01-Sep30_+8_0_-3.xlsx


In [ ]:
# ===============================================
# Database Connection via SOCKS Proxy (No Outline)
# ===============================================

import pymysql
import socks
import socket
from sqlalchemy import create_engine

# -----------------------------------------------
# SOCKS Proxy Configuration
# (Matches DBeaver setup: 127.0.0.1:1080)
# -----------------------------------------------
socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 1080)
socket.socket = socks.socksocket

# -----------------------------------------------
# Database Credentials
# -----------------------------------------------
db_user = "username#"
db_password = "password#"
db_host = "30.123.3.123"
db_port = 1234
db_name = "dbname"

# -----------------------------------------------
# Create SQLAlchemy Engine
# -----------------------------------------------
engine = create_engine(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

# -----------------------------------------------
# Test Connection
# -----------------------------------------------
with engine.connect() as connection:
    print("✅ Connected successfully via SOCKS proxy!")


Connected successfully via SOCKS proxy!


In [16]:
# ==============================================================
# Function: query_incr_multiple
# Description:
#     Fetches game logs for multiple Game Platform IDs (GPIDs)
#     within a specified date range. Automatically adjusts
#     the start and end times to cover the full day (00:00:00–23:59:59).
# ==============================================================

# from datetime import datetime
# import pandas as pd
# import time as tm

def query_incr_multiple(gpid_list, table, start_date, end_date, engine):
    """
    Fetch game logs for multiple game platform IDs within a modified time range.
    
    Args:
        gpid_list (list): List of game platform IDs.
        table (str): Database table name.
        start_date (str): Original start date in "YYYY-MM-DD HH:MM:SS" format.
        end_date (str): Original end date in "YYYY-MM-DD HH:MM:SS" format.
        engine: Database engine connection.
    
    Returns:
        DataFrame: Aggregated query results.
    """
    # -------------------------------------------
    # Convert to datetime and adjust time range
    # -------------------------------------------
    
    # Convert to datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")

    # Modify time portion
    mod_start_dt = start_dt.replace(hour=0, minute=0, second=0)
    mod_end_dt = end_dt.replace(hour=23, minute=59, second=59)

    # Convert back to string 
    mod_start_str = mod_start_dt.strftime("%Y-%m-%d %H:%M:%S")
    mod_end_str = mod_end_dt.strftime("%Y-%m-%d %H:%M:%S")

    # Convert list of GPIDs to a SQL-compatible string
    gpid_list_str = ",".join(map(str, gpid_list))

    # SQL Query: Fetch all GPIDs at once (faster!)
    
    # gl_query = f"""
    # SELECT 
    #     gl.game_platform_id, 
    #     gl.end_at, 
    #     SUM(gl.real_betting_amount) AS Bet, 
    #     SUM(gl.result_amount) AS Payout  
    # FROM game_logs gl
    # JOIN player p ON p.playerId = gl.player_id
    # WHERE gl.end_at BETWEEN '{mod_start_str}' AND '{mod_end_str}' 
    #     AND gl.flag = '1'
    #     AND gl.game_platform_id IN ({gpid_list_str})  -- Fetch all GPIDs in one query
    # GROUP BY gl.game_platform_id;
    # """
    gl_query = f"""
    SELECT 
        gl.game_platform_id, 
        gl.end_at, 
        SUM(gl.real_betting_amount) AS Bet, 
        SUM(gl.result_amount * -1) AS Payout  
    FROM game_logs gl
    JOIN player p ON p.playerId = gl.player_id
    WHERE gl.end_at BETWEEN '{mod_start_str}' AND '{mod_end_str}' 
        AND gl.flag = '1'
        AND gl.game_platform_id IN ({gpid_list_str})  -- Fetch all GPIDs in one query
    GROUP BY gl.game_platform_id;
    """
    # -------------------------------------------
    # Execute query and handle results
    # -------------------------------------------
    
    print("Executing optimized query...\n")
    try:
        # Execute query
        result = pd.read_sql_query(gl_query, engine)
        if result.empty:
            print("No data found. Returning empty DataFrame.")
        return result

    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

print("Done! Result will be printed.")


Done! Result will be printed.


In [17]:
# Run query for multiple GPIDs
df = query_incr_multiple(gpid_list, table, start_date, end_date, engine)

# Display result
#print(df)

Executing optimized query...



In [18]:
# query_start = tm.time()
# query_time = tm.asctime(tm.localtime(query_start))

# # Function to display colored text
# def printmd(string, color="black"):
#     display(Markdown(f"<span style='color:{color}; font-size:16px;'>{string}</span>"))

# # Print in green
# printmd(f"Query Start Run at: {query_time}", "green")

In [19]:
# ============================================
# End Timer and Display Total Execution Time
# ============================================

# Stop timer
query_end = tm.time()
printmd(f"Query End Run at: {tm.asctime(tm.localtime(query_end))}", "red")

# Calculate and print total execution time
execution_time = query_end - query_start
printmd(f"Total Execution Time: {execution_time:.2f} seconds", "blue")

<span style='color:red; font-size:16px;'>Query End Run at: Wed Oct  1 12:45:27 2025</span>

<span style='color:blue; font-size:16px;'>Total Execution Time: 37.14 seconds</span>

In [20]:
#df.to_csv("game_logs_results.csv", index=False)

In [21]:
# ============================================
# Display Formatting for Floating-Point Numbers
# ============================================

# Ensures that all floating-point numbers in DataFrames
# are displayed with exactly 5 decimal places for consistency.

pd.options.display.float_format = '{:.5f}'.format #specifies that floating-point numbers will be displayed with 5 decimal places

In [22]:
print(df)

    game_platform_id              end_at           Bet       Payout
0               5936 2025-09-01 09:26:32   17988.34000   2152.87000
1               6479 2025-09-01 14:00:16   16938.20000  -1797.31000
2               6483 2025-09-01 11:14:14   12979.50000   5375.90000
3               6487 2025-09-03 18:06:17   26941.08200   -110.31200
4               6512 2025-09-06 20:34:51      66.40000    -23.40000
5               6514 2025-09-01 20:41:01    4979.00000    265.50000
6               6515 2025-09-02 22:13:38       7.25000      5.80000
7               6516 2025-09-05 18:43:07      81.10000     10.55600
8               6518 2025-09-01 17:42:15    6303.15000   1904.67000
9               6519 2025-09-01 09:34:03    6605.40000   1294.56000
10              6520 2025-09-04 20:53:18    5454.75000    772.74000
11              6522 2025-09-05 21:18:43    2462.10000     15.13000
12              6523 2025-09-03 04:41:28    1535.50000     46.12000
13              6526 2025-09-01 08:04:10  125451

In [35]:
def get_row_for_gpid(gpid, smashcreds):
    """
    Fetch the row number for a given GPID from the smashcreds dictionary.
    
    Args:
        gpid (str): The game platform ID to look up.
        smashcreds (dict): Dictionary containing GPID-to-row mappings.
    
    Returns:
        str: The row number associated with the given GPID, or None if not found.
    """
    for provider, data in smashcreds.items():
        gpid_row_key = f"{gpid}row"  # Ensure the key matches exactly with the GPID's row
        if gpid_row_key in data:
            return data[gpid_row_key]
    return None  # Return None if the GPID is not found

# Example usage
#gpid_list = ['5940', '5941', '5942', '5943', '6198']
row_mapping = {gpid: get_row_for_gpid(gpid, smashcreds) for gpid in gpid_list}

# Print the row mapping for each GPID
print('Printing row mapping of each GPID')
print(row_mapping)

Printing row mapping of each GPID
{'6540': '2', '6516': '3', '6582': '4', '6479': '5', '6517': '6', '6575': '7', '6519': '8', '5936': '9', '6580': '10', '6578': '11', '6527': '12', '6581': '13', '6577': '14', '6483': '15', '6528': '16', '6520': '17', '6482': '18', '6526': '19', '6515': '20', '6487': '21', '6621': '22', '6512': '23', '6579': '24', '6584': '25', '6518': '26', '6620': '27', '6525': '29', '6622': None, '6524': '31', '6522': '32', '6576': '33', '6529': '34', '6583': '38', '6514': '39', '6513': '40', '6523': '41'}


In [36]:
from openpyxl import load_workbook
import pandas as pd
from shutil import copyfile

# Define the path for the template and the new file
import_template = fr'C:\Users\{pc_username_folder}\automation_project\sorted_folder\Templates\V8 Templates\SMASH_Template_SBE.xlsx'
export_File = fr'C:\Python\smash\smash_{filenamedt}.xlsx'

# Open the existing Excel file for reuse (if it already exists)
def reuse_existing_excel(export_File, df, row_mapping):
    """
    Reuse the existing Excel file and insert the query results into the correct rows.
    
    Args:
        export_File (str): Path to the exported Excel file.
        df (DataFrame): The query result DataFrame.
        row_mapping (dict): A dictionary mapping GPIDs to the row numbers.
    """
    # Open the workbook and select the active sheet
    book = load_workbook(export_File)
    sheet = book.active  # You can specify a sheet name if needed, e.g., sheet = book["Sheet1"]

    # Track the last used row for appending new data (optional)
    last_row = sheet.max_row

    # Insert the results from the DataFrame into the correct columns
    for _, row in df.iterrows():
        gpid = row['game_platform_id']
        bet = row['Bet']
        payout = row['Payout']

        # Get the corresponding row number from the mapping
        row_num = row_mapping.get(str(gpid))

        if row_num:
            # Insert data into columns F (SBE BET) and J (SBE PO)
            sheet[f'F{row_num}'] = bet  # Column F for SBE BET
            sheet[f'J{row_num}'] = payout  # Column J for SBE PO

    # Save the updated workbook without overwriting the template
    book.save(export_File)
    book.close()

# Function to load the template and copy it for the first time if necessary
def create_or_reuse_template():
    # Only copy the template if it doesn't exist yet (for first-time use)
    if not os.path.exists(export_File):
        copyfile(import_template, export_File)
        print(f"Template copied to {export_File}")
    else:
        print(f"Reusing existing file: {export_File}")

# Reuse the existing file or create a new one if it doesn't exist
create_or_reuse_template()

# Insert the results into the reusable file
reuse_existing_excel(export_File, df, row_mapping)

print(f"Results have been successfully saved to: {export_File}")


Reusing existing file: C:\Python\smash\smash_Sep01-Sep30_+8_0_-3.xlsx
Results have been successfully saved to: C:\Python\smash\smash_Sep01-Sep30_+8_0_-3.xlsx


In [37]:
gpid_list =  ['crash','dice','double'] #we use the game to the loop function

In [38]:
def get_row_for_gpid(gpid, smashcreds):
    """
    Fetch the row number for a given GPID from the smashcreds dictionary.
    
    Args:
        gpid (str): The game platform ID to look up.
        smashcreds (dict): Dictionary containing GPID-to-row mappings.
    
    Returns:
        str: The row number associated with the given GPID, or None if not found.
    """
    for provider, data in smashcreds.items():
        gpid_row_key = f"{gpid}row"  # Ensure the key matches exactly with the GPID's row
        if gpid_row_key in data:
            return data[gpid_row_key]
    return None  # Return None if the GPID is not found

# Example usage
#gpid_list = ['5940', '5941', '5942', '5943', '6198']
row_mapping = {gpid: get_row_for_gpid(gpid, smashcreds) for gpid in gpid_list}

# Print the row mapping for each GPID
print('Printing row mapping of each game')
print(row_mapping)

Printing row mapping of each game
{'crash': '35', 'dice': '36', 'double': '37'}


In [39]:
# from datetime import datetime
# import pandas as pd
# import time as tm

def query_incr_multiple(gpid_list, table, start_date, end_date, engine):
    """
    Fetch game logs for multiple game platform IDs within a modified time range.
    
    Args:
        gpid_list (list): List of game platform IDs.
        table (str): Database table name.
        start_date (str): Original start date in "YYYY-MM-DD HH:MM:SS" format.
        end_date (str): Original end date in "YYYY-MM-DD HH:MM:SS" format.
        engine: Database engine connection.
    
    Returns:
        DataFrame: Aggregated query results.
    """

    # Convert to datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")

    # # Modify time portion
    mod_start_dt = start_dt.replace(hour=0, minute=0, second=0)
    mod_end_dt = end_dt.replace(hour=23, minute=59, second=59)

    # # Convert back to string
    mod_start_str = mod_start_dt.strftime("%Y-%m-%d %H:%M:%S")
    mod_end_str = mod_end_dt.strftime("%Y-%m-%d %H:%M:%S")

    # Convert list of GPIDs to a SQL-compatible string
    #gpid_list_str = ",".join(map(str, gpid_list)) #used in gameplatform_id
    gpid_list_str = ",".join(f"'{game}'" for game in gpid_list) #used for gpid_list =  ['crash','dice','double'] #we use the game to the loop function

    # SQL Query: Fetch all GPIDs at once (faster!)
    gl_query = f"""
    SELECT 
        gl.game, 
        gl.end_at, 
        SUM(gl.real_betting_amount) AS Bet, 
        SUM(gl.result_amount * -1) AS Payout 
        # gl.game_platform_id
    FROM game_logs gl
    JOIN player p ON p.playerId = gl.player_id
    WHERE gl.end_at BETWEEN '{mod_start_str}' AND '{mod_end_str}' 
        AND gl.flag = '1'
        AND gl.game_platform_id = '5928'  -- Fetch all GPIDs in one query -in this case fetch all is not applicable we hardcoded the gpid and use gamtypeID
        AND gl.game IN ({gpid_list_str})
    GROUP BY gl.game;
    """

    print("Executing optimized query...\n")
    try:
        # Execute query
        result = pd.read_sql_query(gl_query, engine)
        if result.empty:
            print("No data found. Returning empty DataFrame.")
        return result

    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

In [40]:
# Run query for multiple GPIDs
df = query_incr_multiple(gpid_list, table, start_date, end_date, engine)

# Display result
print(df)

Executing optimized query...

     game              end_at            Bet       Payout
0   crash 2025-09-01 00:00:18 17549921.55640 589905.86029
1    dice 2025-09-01 00:00:07  3285067.69400  89176.54253
2  double 2025-09-01 00:00:12  4040220.80550 302283.88110


In [41]:
engine.dispose()  # Properly closes the database connection

In [42]:
#End timer
query_end = tm.time()
printmd(f"Query End Run at: {tm.asctime(tm.localtime(query_end))}", "red")

# Calculate and print total execution time
execution_time = query_end - query_start
printmd(f"Total Execution Time: {execution_time:.2f} seconds", "blue")

<span style='color:red; font-size:16px;'>Query End Run at: Wed Oct  1 12:45:46 2025</span>

<span style='color:blue; font-size:16px;'>Total Execution Time: 56.01 seconds</span>

In [43]:
from openpyxl import load_workbook
import pandas as pd
from shutil import copyfile

# Define the path for the template and the new file
import_template = fr'C:\Users\{pc_username_folder}\automation_project\sorted_folder\Templates\V8 Templates\SMASH_Template_SBE.xlsx'
export_File = fr'C:\Python\smash\smash_{filenamedt}.xlsx'

# Open the existing Excel file for reuse (if it already exists)
def reuse_existing_excel(export_File, df, row_mapping):
    """
    Reuse the existing Excel file and insert the query results into the correct rows.
    
    Args:
        export_File (str): Path to the exported Excel file.
        df (DataFrame): The query result DataFrame.
        row_mapping (dict): A dictionary mapping GPIDs to the row numbers.
    """
    # Open the workbook and select the active sheet
    book = load_workbook(export_File)
    sheet = book.active  # You can specify a sheet name if needed, e.g., sheet = book["Sheet1"]

    # Track the last used row for appending new data (optional)
    last_row = sheet.max_row

    # Insert the results from the DataFrame into the correct columns
    for _, row in df.iterrows():
        gpid = row['game']
        bet = row['Bet']
        payout = row['Payout']

        # Get the corresponding row number from the mapping
        row_num = row_mapping.get(str(gpid))

        if row_num:
            # Insert data into columns F (SBE BET) and J (SBE PO)
            sheet[f'F{row_num}'] = bet  # Column F for SBE BET
            sheet[f'J{row_num}'] = payout  # Column J for SBE PO

    # Save the updated workbook without overwriting the template
    book.save(export_File)
    book.close()

# Function to load the template and copy it for the first time if necessary
def create_or_reuse_template():
    # Only copy the template if it doesn't exist yet (for first-time use)
    if not os.path.exists(export_File):
        copyfile(import_template, export_File)
        print(f"Template copied to {export_File}")
    else:
        print(f"Reusing existing file: {export_File}")

# Reuse the existing file or create a new one if it doesn't exist
create_or_reuse_template()

# Insert the results into the reusable file
reuse_existing_excel(export_File, df, row_mapping)

print(f"Results have been successfully saved to: {export_File}")


Reusing existing file: C:\Python\smash\smash_Sep01-Sep30_+8_0_-3.xlsx
Results have been successfully saved to: C:\Python\smash\smash_Sep01-Sep30_+8_0_-3.xlsx
